# AutoGen 기본 샘플

이 코드 샘플에서는 [AutoGen](https://aka.ms/ai-agents/autogen) AI 프레임워크를 사용하여 기본 에이전트를 생성합니다.

이 샘플의 목표는 다양한 에이전트 패턴을 구현할 때 추가 코드 샘플에서 사용할 단계들을 보여주는 것입니다.


## 필요한 Python 패키지 가져오기


In [ ]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


## 클라이언트 생성하기

이 샘플에서는 LLM에 접근하기 위해 [GitHub Models](https://aka.ms/ai-agents-beginners/github-models)를 사용할 것입니다.

`model`은 `gpt-4o-mini`로 정의되어 있습니다. GitHub Models 마켓플레이스에서 제공되는 다른 모델로 변경하여 다양한 결과를 확인해 보세요.

간단한 테스트로, `프랑스의 수도는 어디인가요?`라는 간단한 프롬프트를 실행해 보겠습니다.


In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## 에이전트 정의하기

`client`를 설정하고 정상적으로 작동하는 것을 확인했으니, 이제 `AssistantAgent`를 만들어 보겠습니다. 각 에이전트는 다음과 같은 속성을 가질 수 있습니다:
**name** - 여러 에이전트가 함께 작동하는 흐름에서 참조하기에 유용한 간단한 이름.
**model_client** - 이전 단계에서 생성한 클라이언트.
**tools** - 에이전트가 작업을 완료하기 위해 사용할 수 있는 도구들.
**system_message** - LLM의 작업, 행동 및 톤을 정의하는 메타프롬프트.

시스템 메시지를 변경하여 LLM의 반응을 확인할 수 있습니다. `tools`에 대해서는 Lesson #4에서 다룰 예정입니다.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are a travel agent that plans great vacations",
)

## 에이전트 실행하기

아래 함수는 에이전트를 실행합니다. 우리는 `on_message` 메서드를 사용하여 새로운 메시지로 에이전트의 상태를 업데이트합니다.

이 경우, 사용자의 새로운 메시지인 `"Plan me a great sunny vacation"`으로 상태를 업데이트합니다.

메시지 내용을 변경하여 LLM이 어떻게 다르게 반응하는지 확인할 수 있습니다.


In [ ]:
from IPython.display import display, HTML


async def assistant_run():
    # Define the query
    user_query = "Plan me a great sunny vacation"

    # Start building HTML output
    html_output = "<div style='margin-bottom:10px'>"
    html_output += "<div style='font-weight:bold'>User:</div>"
    html_output += f"<div style='margin-left:20px'>{user_query}</div>"
    html_output += "</div>"

    # Execute the agent response
    response = await agent.on_messages(
        [TextMessage(content=user_query, source="user")],
        cancellation_token=CancellationToken(),
    )

    # Add agent response to HTML
    html_output += "<div style='margin-bottom:20px'>"
    html_output += "<div style='font-weight:bold'>Assistant:</div>"
    html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.chat_message.content}</div>"
    html_output += "</div>"

    # Display formatted HTML
    display(HTML(html_output))

# Run the function
await assistant_run()


---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전을 신뢰할 수 있는 권위 있는 자료로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 당사는 책임을 지지 않습니다.
